In [127]:
import mygrad as mg
#loss = mg.mean(mg.maximum(0, margin - y * (x1 - x2)))
import numpy as np
from gensim.models import KeyedVectors
from cogworks_data.language import get_data_path
from pathlib import Path
import json
import re, string
from collections import Counter
import numpy as np
import math
import pickle

In [1]:
# load COCO metadata
filename = get_data_path("captions_train2014.json")
with Path(filename).open() as f:
    coco_data = json.load(f)

In [9]:
filename = "glove.6B.200d.txt.w2v"

# this takes a while to load -- keep this in mind when designing your capstone project
glove = KeyedVectors.load_word2vec_format(get_data_path(filename), binary=False)

In [83]:
punc_regex = re.compile('[{}]'.format(re.escape(string.punctuation)))
def strip_punc(corpus):
    return punc_regex.sub('', corpus)

text = "Apples rule. Apples are the best. Truly, they are. Truly... Truly"
text = strip_punc(text).lower()
tokenized = text.split()
tokens = Counter(tokenized)
#descriptor = np.array([tokens[word] for word in sorted(tokens)], dtype=float)
#descriptor
#tokens

**Embed Caption**

In [16]:
def to_counter(doc):
    return Counter(strip_punc(doc).lower().split())

In [40]:
def to_vocab(counters, k=None):
    output = set()
    for i in counters:
        output.add(i)
    #output = set(output)
    return sorted(output)

In [18]:
def to_idf(vocab, counters):
    N = len(counters)
    nt = [sum(1 if term in counter else 0 for counter in counters) for term in vocab]
    nt = np.array(nt, dtype=float)

    return np.log10(N/nt)    

In [159]:
def embed_caption(caption):
    caption = to_counter(caption)
    caption_idf = to_idf(to_vocab(caption), caption)
    #caption_glove = [glove[i] for i in caption]
    caption_glove=[]
    for i in range(len(caption)):
        try:
            caption_glove.append(glove[caption[i]])
        except:
            caption_idf.pop(i)
            continue
    
    dots=[]
    for i in range(len(caption_idf)):
        dots.append(np.dot(caption_idf[i], caption_glove[i]))        
    summation = sum(dots)
    unit = summation / math.sqrt(sum(summation**2))

    return unit

**Image Descriptor**

In [200]:
with Path(get_data_path('resnet18_features.pkl')).open('rb') as f:
    resnet18_features = pickle.load(f)

In [120]:
resnet18_features[57870].shape

(1, 512)

In [193]:
coco_data["annotations"][0]

{'image_id': 318556,
 'id': 48,
 'caption': 'A very clean and well decorated empty bathroom'}

**Creating Training Data**

In [167]:
len(resnet18_features)

82612

In [164]:
len(coco_data["images"])

82783

In [221]:
training_data = []
caption_vectors = []
good_image_vectors = []
bad_image_vectors = []
index=0

for i in range(len(list(resnet18_features.keys())[:500])):
    caption_vectors.append(embed_caption(coco_data["annotations"][i]["caption"]))
    good_image_vectors.append(resnet18_features[list(resnet18_features.keys())[i]])
    """
    while True:
        random_key = list(resnet18_features.keys())[np.random.randint(len(resnet18_features.keys()))]
        if (random_key!=coco_data["annotations"][j]["image_id"]):
            break
    bad_image_vectors.append(resnet18_features[random_key])
    """
    random_key = list(resnet18_features.keys())[np.random.randint(len(resnet18_features.keys()))]
    if (random_key!=coco_data["annotations"][i]["image_id"]):
        bad_image_vectors.append(resnet18_features[random_key])
        training_data.append((coco_data["annotations"][i]["id"], coco_data["annotations"][i]["image_id"], random_key)) #(true-caption-ID, true-image-ID, confusor-image-ID)


In [220]:
training_data

[(48, 318556, 519407),
 (67, 116100, 548913),
 (126, 318556, 16543),
 (148, 116100, 263759),
 (173, 379340, 252604),
 (188, 379340, 280740),
 (219, 318556, 80431),
 (255, 318556, 381400),
 (272, 134754, 287098),
 (288, 538480, 12673),
 (314, 476220, 329001),
 (328, 299675, 101742),
 (352, 32275, 394929),
 (411, 302443, 91172),
 (425, 134754, 41340),
 (519, 25470, 241461),
 (572, 513461, 424915),
 (653, 18691, 211735),
 (668, 285579, 407751),
 (674, 266366, 232379),
 (710, 226658, 266586),
 (715, 299675, 456528),
 (729, 124567, 573956),
 (750, 293605, 421648),
 (760, 321194, 362152),
 (764, 479495, 344637),
 (818, 539984, 82313),
 (819, 28231, 296289),
 (873, 538480, 354863),
 (877, 350235, 480474),
 (928, 314265, 514213),
 (946, 389006, 58497),
 (1022, 134754, 72905),
 (1029, 31813, 157921),
 (1098, 538480, 59995),
 (1124, 465049, 136154),
 (1161, 360334, 367449),
 (1184, 524679, 487467),
 (1193, 279672, 266600),
 (1199, 385716, 288471),
 (1231, 44816, 546114),
 (1273, 116100, 160402),

**Model**

In [126]:
#caption vector
#embed_caption(coco_data["annotations"]["48"])]
coco_data["annotations"][0]

{'image_id': 318556,
 'id': 48,
 'caption': 'A very clean and well decorated empty bathroom'}

In [ ]:
#descriptor vectors
resnet18_features[57870]